In [ ]:
# See README for more info on how the DataCollectionPipeline works
# The ETL pipeline is part of the DataCollectionPipeline
# Remove the time.sleep(1) line if you are sure you won't get blocked from a webpage for requesting too often
import requests
from bs4 import BeautifulSoup
import time
import os
import shutil
import subprocess
import tempfile
import urllib.parse
from shared import getMongoClient

# Input into the Data Collection Pipeline is a list of links to domains
links = ['https://www.ros.org/','https://docs.nav2.org/','https://moveit.ai/','https://gazebosim.org/home', 'https://github.com/ros2/ros2', 'https://github.com/ros-navigation/navigation2', 'https://github.com/moveit/moveit2', 'https://github.com/gazebosim/gazebo-classic']

# Create a mongoDB connection
mongoHost =  getMongoClient()
mongoDatabase =  mongoHost["twin"]
# ETL pipeline
# Extract data from links and their subdirectories(using crawlers)
codes = []
for link in links:
    # Web scraper/crawler for github links
    if "https://github.com" in link:
        # Do not revisit a link already in the database
        mongoCollection = mongoDatabase["Github"]
        result = mongoCollection.find_one({"link": {"$regex" : link}})
        if result is None and link:
            print("Visiting link: ", link)
            # Modified GithubCrawler from LLM-Engineer for scraping github
            local_temp = tempfile.mkdtemp()
            try:
                os.chdir(local_temp)
                subprocess.run(["git", "clone", link])
                repo_path = os.path.join(local_temp, os.listdir(local_temp)[0])
                tree = {}
                for root, _, files in os.walk(repo_path):
                    dir = root.replace(repo_path, "").lstrip("/")
                    if dir.startswith((".git", ".toml", ".lock", ".png")):
                        continue
                    for file in files:
                        if file.endswith((".git", ".toml", ".lock", ".png")):
                            continue
                        file_path = os.path.join(dir, file)
                        with open(
                            os.path.join(root, file), "r", errors="ignore"
                        ) as f:
                            tree[file_path] = f.read().replace(" ", "")
            except Exception:
                print(f"Error scrapping {link}")
            finally:
                shutil.rmtree(local_temp)
                # Correct the link
                r = requests.get(link)
                soup = BeautifulSoup(r.content, "html.parser")
                # Find the file path to any of the files in the repository
                link_element = soup.find("a", attrs={"class": "Link--primary"})
                path = link_element.get("href")
                path = path.rsplit("/", 1)[0]
                # Push all the subdirectories to mongo
                for subdirectory in tree:
                    #print(
                    #    f"Adding subdirectory: https://github.com{path}/{subdirectory}"
                    #)
                    text = tree[subdirectory]
                    # Transform the data
                    # Get rid of repeating \n characters and spaces
                    text = text.replace("\t", " ")
                    text = text.replace("\n", " ")
                    text_len = len(text)
                    for i in range(text_len):
                        while i + 1 < text_len and text[i] == " " and text[i + 1] == " ":
                            text = text[:i] + text[i + 1 :]
                            text_len -= 1
                    codes.append(
                        {
                            "link": "https://github.com"
                            + path
                            + "/"
                            + subdirectory,
                            "type": "Github",
                            "content": text,
                        }
                    )
                mongoCollection.insert_many(codes)
                codes = []
        else:
            print("Already visited: ", link)
    # Web scraper/crawler for other links(Documents)
    else:
        # Do not revisit a link already in the database
        mongoCollection = mongoDatabase["Document"]
        result = mongoCollection.find_one({"link": link})
        if result is None:
            print("Visiting link: ", link)
            try:
                # Get all text in the website
                r = requests.get(link)
                soup = BeautifulSoup(r.content, "html.parser")
                soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6"])
                text = soup.get_text()
                # Transform the data
                # Get rid of repeating \n characters and spaces
                text = text.replace("\t", " ")
                text = text.replace("\n", " ")
                text_len = len(text)
                for i in range(text_len):
                    while i + 1 < text_len and text[i] == " " and text[i + 1] == " ":
                        text = text[:i] + text[i + 1 :]
                        text_len -= 1
                if "404" not in text:
                    mongoCollection.insert_one({"link": link, "type": "Document", "content": text})
                else:
                    print("Page not found: ", link)
                # Also crawl through all subdirectorys in the link(related links)
                soup = BeautifulSoup(r.content, "html.parser")
                subdirectories = [a.get("href") for a in soup.find_all("a")]
                for subdirectory in subdirectories:
                    newLink = urllib.parse.urljoin(link, subdirectory)
                    if (
                        subdirectory is not None and
                        'http' not in subdirectory and
                        '#' not in subdirectory and
                        mongoCollection.find_one({"link": newLink}) is None and
                        newLink not in links
                    ):
                        #print("Adding subdirectory: ", link + subdirectory)
                        links.append(newLink)
            except:
                print("Could not crawl link", link)
        else:
            print("Already visited: ", link)
    # Avoid spamming sites
    time.sleep(0.1)

Visiting link:  https://www.ros.org/
Visiting link:  https://docs.nav2.org/
Visiting link:  https://moveit.ai/
Visiting link:  https://gazebosim.org/home
Visiting link:  https://github.com/ros2/ros2


Cloning into 'ros2'...


Visiting link:  https://github.com/ros-navigation/navigation2


Cloning into 'navigation2'...


Visiting link:  https://github.com/moveit/moveit2


...

Page not found:  https://moveit.ai/events/rosworld-2021-workshop/MoveIt%20and%20the%20rest%20of%20ROS_%20Perception,%20Control,%20and%20Simulation%20ROSWorld%20October%202021%20-%20Mobile%20Manipulation%20Workshop.pdf
Visiting link:  https://moveit.ai/events/rosworld-2021-workshop/ROSWorld%202021%20-%20Mobile%20Manipulation%20Workshop%20-%20MoveIt%20Task%20Constructor.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Page not found:  https://moveit.ai/events/rosworld-2021-workshop/ROSWorld%202021%20-%20Mobile%20Manipulation%20Workshop%20-%20MoveIt%20Task%20Constructor.pdf
Visiting link:  https://moveit.ai/events/rosworld-2021-workshop/#expected-outcomes
Visiting link:  https://moveit.ai/events/rosworld-2021-workshop/#preparation-for-the-workshop
Visiting link:  https://moveit.ai/events/rosworld-2021-workshop/#installation-instructions
Visiting link:  https://moveit.ai/events/rosworld-2021-workshop/#further-reading--watching
Visiting link:  https://moveit.ai/events/rosworld-2021-workshop/Stretch%20Payload%20&%20Pulling%20Force.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visiting link:  https://moveit.ai/events/2022-moveit-community-meeting/#time-and-venue
Visiting link:  https://moveit.ai/events/2022-moveit-community-meeting/#organizers
Visiting link:  https://moveit.ai/events/2022-moveit-community-meeting/#program
Visiting link:  https://moveit.ai/events/2022-moveit-community-meeting/#registration
Visiting link:  https://moveit.ai/events/world-moveit-day-2018/#date
Visiting link:  https://moveit.ai/events/world-moveit-day-2018/#description
Visiting link:  https://moveit.ai/events/world-moveit-day-2018/#locations
Visiting link:  https://moveit.ai/events/world-moveit-day-2018/#general-information-contacts
Visiting link:  https://moveit.ai/events/world-moveit-day-2018/#signup
Visiting link:  https://moveit.ai/events/world-moveit-day-2018/#logistics
Visiting link:  https://moveit.ai/events/world-moveit-day-2018/#what-version-of-moveit-should-you-use
Visiting link:  https://moveit.ai/events/world-moveit-day-2018/#finding-where-you-can-help
Visiting link: 